By hard transfer we mean using the model training on the data from IPSL as is to infer for ERA5

In [1]:
import numpy as np
import tensorflow as tf
import xarray as xr
import keras
from keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from unet_ipsl_psl_zg_pr10_tas_to_tas_lag1_COMMITOR import create_UNET, CustomDataset  # import first script

2025-09-15 10:23:12.479809: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-15 10:23:12.499167: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-15 10:23:12.499184: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-15 10:23:12.499772: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-15 10:23:12.503070: I tensorflow/core/platform/cpu_feature_guar

In [7]:
pretrained_model_path = "models/psl_zg_pr10_tas_to_tas_ipsl_lag1_COMMITTOR.keras"
base_model_ipsl = keras.models.load_model(pretrained_model_path, compile=True)
pretrained_model_path = "models/psl_zg_pr10_tas_to_tas_era5_lag1_COMMITTOR.keras"
base_model_era5 = keras.models.load_model(pretrained_model_path, compile=True)
pretrained_model_path = "models/temp.keras"
base_model_era5_transfer = keras.models.load_model(pretrained_model_path, compile=True)

# New dataset (same preprocessing!)

zg_file_train = "/homedata/pchevali/ERA5/training_val_sets/zg_anom_era5_train.nc"
psl_file_train = "/homedata/pchevali/ERA5/training_val_sets/psl_anom_era5_train.nc"
pr10_file_train = "/homedata/pchevali/ERA5/training_val_sets/pr10_anom_era5_train.nc"
tas_file_train = "/homedata/pchevali/ERA5/training_val_sets/tas_anom_era5_train.nc"

zg_file_val = "/homedata/pchevali/ERA5/training_val_sets/zg_anom_era5_val.nc"
psl_file_val = "/homedata/pchevali/ERA5/training_val_sets/psl_anom_era5_val.nc"
pr10_file_val = "/homedata/pchevali/ERA5/training_val_sets/pr10_anom_era5_val.nc"
tas_file_val = "/homedata/pchevali/ERA5/training_val_sets/tas_anom_era5_val.nc"

# Train and val datasets
dataset_train = CustomDataset(
    psl_file_train, zg_file_train, pr10_file_train, tas_file_train, batch_size=32, lag=1
)

dataset_val = CustomDataset(
    psl_file_val, zg_file_val, pr10_file_val, tas_file_val,
    batch_size=32, lag=1,
    std_ref_psl=dataset_train.psl_std,
    std_ref_zg=dataset_train.zg_std,
    std_ref_pr10=dataset_train.pr10_std
)


In [8]:
base_model_ipsl.evaluate(dataset_val)

/net/nfs/tools/u20/Python/miniconda3_py311_23.11.0-2/envs/tensorflow-2.15.0/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9920 - loss: 0.0208


[0.022709663957357407, 0.9913025498390198]

In [9]:
base_model_era5.evaluate(dataset_val)

62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9890 - loss: 0.0258


[0.027813488617539406, 0.9880977869033813]

In [10]:
base_model_era5_transfer.evaluate(dataset_val)

62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9898 - loss: 0.0072


[0.0077635678462684155, 0.9891030788421631]

In [11]:
out_predict = np.squeeze(base_model_ipsl.predict(dataset_val))
out_true = np.concatenate([y for _, y in dataset_val], axis=0)
out_true = np.squeeze(out_true)

time = dataset_val.tas_data.time.values[:out_true.shape[0]]
lat = dataset_val.tas_data.lat.values
lon = dataset_val.tas_data.lon.values

ds = xr.Dataset(
    {
        "out_true": (["time", "lat", "lon"], out_true),
        "out_predict": (["time", "lat", "lon"], out_predict),
    },
    coords={"time": time, "lat": lat, "lon": lon}
)
ds.to_netcdf("models/psl_zg_pr10_tas_to_tas_hard_transfer_lag1_COMMITTOR.nc")

62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
